In [102]:
# import nltk
import requests
from bs4 import BeautifulSoup
# import string
from fake_useragent import UserAgent
import lxml.html as lh
# import newspaper
# import random
import pandas as pd

In [92]:
ua = UserAgent()
header = {'User-Agent':ua.chrome}
session = requests.Session()
base_url = r"http://cba.sports.sina.com.cn/cba/stats/playerstats/"
response = session.get(base_url,headers = header)

response.encoding = 'GBK'

In [93]:
doc = lh.fromstring(req.content)

In [94]:
rows = doc.xpath('//tr')

In [95]:
table_headers = [t.text_content() for t in rows[0]]
print(table_headers)

['号码', '球员', '球队', '场次', '得分', '二分', '三分', '罚球', '篮板', '前篮板', '后篮板', '助攻', '抢断', '扣篮', '盖帽', '失误', '犯规', '效率']


In [68]:
len(tr_elements)

371

In [100]:
i=0
table = []
# 
for row in rows:
    one_row = []
    for cell in row:
        txt=cell.text_content().strip()
        one_row.append(txt)
    table.append(one_row)

In [99]:
table

[['号码',
  '球员',
  '球队',
  '场次',
  '得分',
  '二分',
  '三分',
  '罚球',
  '篮板',
  '前篮板',
  '后篮板',
  '助攻',
  '抢断',
  '扣篮',
  '盖帽',
  '失误',
  '犯规',
  '效率'],
 ['1',
  '琼斯',
  '吉林',
  '20',
  '38.0',
  '10.1-17.6',
  '2.7-9.3',
  '9.8-12.1',
  '8.0',
  '2.1',
  '6.0',
  '8.5',
  '1.7',
  '0.2',
  '0.5',
  '3.5',
  '3.9',
  '36.85'],
 ['2',
  '布莱克尼',
  '江苏',
  '4',
  '36.3',
  '9.8-18.3',
  '3.3-7.0',
  '7.0-9.8',
  '6.5',
  '0.5',
  '6.0',
  '3.3',
  '1.5',
  '1.3',
  '1.0',
  '4.3',
  '2.0',
  '29.25'],
 ['3',
  '约瑟夫-杨',
  '同曦',
  '20',
  '35.7',
  '9.4-16.1',
  '3.6-8.7',
  '6.1-6.8',
  '4.4',
  '0.6',
  '3.8',
  '6.8',
  '1.7',
  '0.8',
  '0.0',
  '3.8',
  '1.4',
  '32.25'],
 ['4',
  '亚当斯',
  '青岛',
  '19',
  '35.4',
  '8.0-13.5',
  '3.8-11.8',
  '8.1-9.2',
  '8.4',
  '0.9',
  '7.5',
  '5.5',
  '2.6',
  '0.1',
  '0.3',
  '4.1',
  '2.3',
  '33.37']]

In [101]:
len(table)

371

In [107]:
player_stats_sina = pd.DataFrame(table[1:],columns = table[0])
player_stats_sina

,号码,球员,球队,场次,得分,二分,三分,罚球,篮板,前篮板,后篮板,助攻,抢断,扣篮,盖帽,失误,犯规,效率
0,1,琼斯,吉林,20,38.0,10.1-17.6,2.7-9.3,9.8-12.1,8.0,2.1,6.0,8.5,1.7,0.2,0.5,3.5,3.9,36.85
1,2,布莱克尼,江苏,4,36.3,9.8-18.3,3.3-7.0,7.0-9.8,6.5,0.5,6.0,3.3,1.5,1.3,1.0,4.3,2.0,29.25
2,3,约瑟夫-杨,同曦,20,35.7,9.4-16.1,3.6-8.7,6.1-6.8,4.4,0.6,3.8,6.8,1.7,0.8,0.0,3.8,1.4,32.25
3,4,亚当斯,青岛,19,35.4,8.0-13.5,3.8-11.8,8.1-9.2,8.4,0.9,7.5,5.5,2.6,0.1,0.3,4.1,2.3,33.37
4,5,德兹明-威尔斯,广厦,1,34.0,10.0-19.0,2.0-5.0,8.0-12.0,11.0,0.0,11.0,4.0,3.0,0.0,1.0,6.0,1.0,31.00
5,6,汉斯布鲁,四川,7,33.1,10.0-18.3,0.3-1.4,12.3-14.0,13.7,4.9,8.9,0.9,2.3,2.1,0.7,1.7,3.3,37.86
6,7,吉布森,江苏,13,32.2,4.5-9.8,4.6-10.0,9.2-10.7,5.3,1.0,4.3,4.2,1.2,0.0,0.1,2.8,3.2,28.08
7,8,福特森,四川,20,29.6,6.6-12.6,2.5-7.0,9.0-12.0,5.4,1.5,4.0,7.6,1.7,0.0,0.0,3.6,3.8,27.20
8,9,富兰克林,山西,18,29.6,6.3-10.9,2.9-8.1,8.3-9.8,9.0,1.6,7.4,10.4,2.6,0.4,1.7,3.8,2.8,38.28
9,10,凯弗-赛克斯,广州,15,28.2,8.3-15.3,2.1-6.5,5.1-5.7,5.3,1.3,4.1,6.1,1.5,0.5,0.1,2.5,2.1,26.87
